**Initialising ORKG for synthesis**

In [ ]:
# Install latest version of the ORKG R package from Git
install.packages('remotes')
remotes::install_gitlab("TIBHannover/orkg/orkg-r")

In [7]:
# Connect with ORKG
library(orkg)
orkg <- ORKG(host='https://incubating.orkg.org/')

In [9]:
# Retrieve resouce from ORKG
PerezAlvarez2018 <- orkg$resources$by_id('R535310')$as_dataframe()
PerezAlvarez2018

,p-value,t-value,DF,Std.Error,Value
,<chr>,<chr>,<chr>,<chr>,<chr>
(Intercept),0.5768,0.5668,21,0.0791,0.0449
mead_250,0.0054,3.1029,21,0.1866,0.5791
Year2015,0,5.5384,21,0.0582,0.3225


In [32]:
# Example
mead_250_P <- as.numeric(PerezAlvarez2018['mead_250',]['p-value'])
mead_250_P

[1] 0.0054